In [1]:
import sqlite3
import pandas as pd
import os
from mido import MidiFile
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
# import musicbrainzngs

os.chdir('../')

from sms.defaults import *

import magenta.music as mm
from note_seq.midi_io import note_sequence_to_midi_file
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from sms.src.embeddings.music_vae.two_bar_encoder import Mel2barEncoderDefault
from sms.src.embeddings.music_vae.two_bar_encoder import Mel2barEncoderCustom
from logging import getLogger
import logging

logging.basicConfig(level=logging.ERROR)
logger = getLogger(__name__)

c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find 

Instructions for updating:
non-resource variables are not supported in the long term


In [9]:
from sms.src.synthetic_data.midi_inserter import MidiInserter
from sms.src.synthetic_data.monophonic_melody_extractor import MonophonicMelodyExtractor
from sms.src.log import configure_logging
import logging

logger = getLogger(__name__)

configure_logging()

path = MONOPHONIC_MIDIS_PATH
path = r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\synthetic_dataset\monophonic_midis\midi_mono"

monophonic_melodies = [os.path.join(root, f) for root, dirs, files in os.walk(path) for f in files if f.endswith('.mid')]

file = monophonic_melodies[2067]
print(file)
# file = r"C:\Users\cunn2\Desktop\midi_databases\aa_midis\No-Scrubs.mid"
# extractor = MonophonicMelodyExtractor()
# extractor.make_file_valid(file, file)

MIDIPlayer(file, height=400, styler=cifka_advanced)


C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\synthetic_dataset\monophonic_midis\midi_mono\NLB072928_01.mid


In [10]:
import mido
bar_extractor = MidiInserter()
dest = 'dest.midi'
bar_extractor.extract_bars(file, dest, 1, 1)
# Print notes in the MIDI file
midi = mido.MidiFile(dest)
notes = []
current_time = 0
for track in midi.tracks:
    for msg in track:
        print(msg)
        current_time += msg.time
MIDIPlayer(dest, height=400, styler=cifka_advanced)


<meta message set_tempo tempo=500000 time=0>
<meta message time_signature numerator=6 denominator=8 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
note_off channel=0 note=69 velocity=64 time=0
note_on channel=0 note=74 velocity=64 time=0
note_off channel=0 note=74 velocity=64 time=60
note_on channel=0 note=71 velocity=64 time=0
<meta message text text='!! segment 1.0' time=300>
note_off channel=0 note=71 velocity=64 time=0
note_on channel=0 note=69 velocity=64 time=0
note_off channel=0 note=69 velocity=64 time=60
note_on channel=0 note=67 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=120
note_on channel=0 note=67 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=60
note_on channel=0 note=67 velocity=64 time=0
<meta message end_of_track time=0>


In [12]:
import mido 
from sms.src.synthetic_data.midi_modifier import MidiModifier, MidiModifierConfig, Note

modified_dest = "modified_dest.midi"

midi = MidiFile(dest) 

midi_modifier = MidiModifier()
available_notes = midi_modifier.identify_note_events(midi)

config_transposition = MidiModifierConfig(
    use_transposition=True,
    transposition_semitone=12
)

config_selected_notes = MidiModifierConfig(
    use_shift_selected_notes_pitch=True,
    selected_notes_pitch_shifts=[(available_notes[0], 30)]
)

config_durations = MidiModifierConfig(
    use_change_note_durations=True,
    note_scale_factors=[(available_notes[5], 5.0)]
)

config_delete_notes = MidiModifierConfig(
    use_delete_notes=True,
    notes_to_delete=[available_notes[0], available_notes[1]]
)

midi_modifier.modify_midi(midi, config_transposition, modified_dest)

midi_modified = MidiFile(modified_dest)

for msg in midi_modified.tracks[0]:
    print(msg)

# Play the modified MIDI
MIDIPlayer(modified_dest, height=400, styler=cifka_advanced)



<meta message set_tempo tempo=645160 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=51 velocity=80 time=0
note_on channel=0 note=51 velocity=0 time=55
note_on channel=0 note=67 velocity=80 time=0
note_on channel=0 note=67 velocity=0 time=55
note_on channel=0 note=70 velocity=80 time=0
note_on channel=0 note=70 velocity=0 time=55
note_on channel=0 note=73 velocity=80 time=0
note_on channel=0 note=73 velocity=0 time=220
note_on channel=0 note=56 velocity=80 time=55
note_on channel=0 note=56 velocity=0 time=55
note_on channel=0 note=68 velocity=80 time=0
note_on channel=0 note=68 velocity=0 time=55
note_on channel=0 note=71 velocity=80 time=0
note_on channel=0 note=71 velocity=0 time=55
note_on channel=0 note=75 velocity=80 time=0
note_on channel=0 note=75 velocity=0 time=220
<meta message end_of_track time=0>


In [5]:
encoder = Mel2barEncoderCustom()

encoded_midis = encoder.encode_midis([dest, modified_dest])

np.linalg.norm(encoded_midis['dest.midi']['0'] - encoded_midis['modified_dest.midi']['0'])



INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



...

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\magenta\models\music_vae\base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


ValueError: The passed save_path is not a valid checkpoint: plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt

In [ ]:
encoder = Mel2barEncoderCustom()

loops = 10
vecs = []
for i in range(loops):
    vecs.append(encoder.encode_midis([dest])['dest.midi']['0'])

vec_mean = np.mean(vecs, axis=0)
encoder.decode_vector(vec_mean, output_path='recon.midi', temperature=1)

MIDIPlayer('recon.midi', height=400, styler=cifka_advanced)


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]



INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


INFO:tensorflow:Restoring parameters from plagdet/models/music_vae/2_bar_melody/cat-mel_2bar_big.ckpt


In [ ]:
a, b = encoded_midis['dest.midi']['0'], encoded_midis['modified_dest.midi']['0']

print(a.shape)
print(b.shape)

# Calculate Euclidean distance between a and b
print(np.linalg.norm(a - b))



(512,)
(512,)
32.247547


In [ ]:
file2 = monophonic_melodies[4]

MIDIPlayer(file2, height=400, styler=cifka_advanced)

In [ ]:
bar_extractor = MidiInserter()

dest2 = 'dest2.midi'

bar_extractor.extract_bars(file2, dest2, 1, 2)

# Print notes in the MIDI file
midi = mido.MidiFile(dest2)
notes = []
current_time = 0

for track in midi.tracks:
    for msg in track:
        print(msg)
        current_time += msg.time



MIDIPlayer(dest2, height=400, styler=cifka_advanced)

<meta message set_tempo tempo=500000 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=77 velocity=0 time=0
note_on channel=0 note=78 velocity=80 time=0
note_on channel=0 note=78 velocity=0 time=330
note_on channel=0 note=69 velocity=80 time=330
note_on channel=0 note=46 velocity=80 time=55
note_on channel=0 note=69 velocity=0 time=0
note_on channel=0 note=46 velocity=0 time=275
note_on channel=0 note=70 velocity=80 time=275
note_on channel=0 note=65 velocity=80 time=55
note_on channel=0 note=70 velocity=0 time=0
note_on channel=0 note=65 velocity=0 time=220
<meta message end_of_track time=0>


In [ ]:
encoded_midi = encoder.encode_midis([dest2])


In [ ]:
c = encoded_midi['dest2.midi']['0']

print(np.linalg.norm(a-c))

print(np.dot(a, c) / (np.linalg.norm(a) * np.linalg.norm(c)))

32.820896
0.021056492


In [ ]:
type(a)

numpy.ndarray

In [ ]:
encoder.decode_vector(b, output_path='recon.midi')

MIDIPlayer('recon.midi', height=400, styler=cifka_advanced)


In [ ]:
recon2 = mel_2bar.decode(b, length=50)[0]

MIDIPlayer(recon2, height=400, styler=cifka_advanced)

NameError: name 'mel_2bar' is not defined

In [ ]:

recon3 = mel_2bar.decode(c, length=50)[0]

MIDIPlayer(recon3, height=400, styler=cifka_advanced)